<a href="https://colab.research.google.com/github/inokchoi/ComputerVision/blob/main/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-

"""
CIFAR-10 Convolutional Neural Networks(CNN) Example
next_batch function is copied from edo's answer
https://stackoverflow.com/questions/40994583/how-to-implement-tensorflows-next-batch-for-own-data
Author : solaris33
Project URL : http://solarisailab.com/archives/2325
"""

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# CIFAR-10 데이터를 다운로드 받기 위한 keras의 helper 함수인 load_data 함수를 임포트합니다.
from tensorflow.keras.datasets.cifar10 import load_data


In [ ]:
# 다음 배치를 읽어오기 위한 next_batch 유틸리티 함수를 정의합니다.
def next_batch(num, data, labels):
    '''
    `num` 개수 만큼의 랜덤한 샘플들과 레이블들을 리턴합니다.
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [ ]:
# CNN 모델을 정의합니다. 
def build_CNN_classifier(x):
    # 입력 이미지
    x_image = x

    weight_decay = 1e-4

    # 첫번째 convolutional layer - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)합니다.
    #W_conv1 = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 32], stddev=5e-2))
    W_conv1 = tf.get_variable("W_conv1", shape=[5, 5, 3, 32],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
    h_bn1 = tf.layers.batch_normalization(h_conv1)

    #W_conv1_2 = tf.Variable(tf.truncated_normal(shape=[3, 3, 32, 32], stddev=5e-2))
    W_conv1_2 = tf.get_variable("W_conv1_2", shape=[5, 5, 32, 32],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv1_2 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1_2 = tf.nn.elu(tf.nn.conv2d(h_bn1, W_conv1_2, strides=[1, 1, 1, 1], padding='SAME') + b_conv1_2)
    h_bn1_2 = tf.layers.batch_normalization(h_conv1_2)

    W_conv1_3 = tf.get_variable("W_conv1_3", shape=[5, 5, 32, 32],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv1_3 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1_3 = tf.nn.elu(tf.nn.conv2d(h_bn1_2, W_conv1_3, strides=[1, 1, 1, 1], padding='SAME') + b_conv1_3)
    h_bn1_3 = tf.layers.batch_normalization(h_conv1_3)

    # 첫번째 Pooling layer
    h_pool1 = tf.nn.max_pool(h_bn1_3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    h_pool1_drop = tf.nn.dropout(h_pool1, keep_prob) 


    # 두번째 convolutional layer - 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)합니다.
    #W_conv2 = tf.Variable(tf.truncated_normal(shape=[3, 3, 32, 64], stddev=5e-2))
    W_conv2 = tf.get_variable("W_conv2", shape=[3, 3, 32, 64],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.elu(tf.nn.conv2d(h_pool1_drop, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
    h_bn2 = tf.layers.batch_normalization(h_conv2)

    #W_conv2_2 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 64], stddev=5e-2))
    W_conv2_2 = tf.get_variable("W_conv2_2", shape=[3, 3, 64, 64],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv2_2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2_2 = tf.nn.elu(tf.nn.conv2d(h_bn2, W_conv2_2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2_2)
    h_bn2_2 = tf.layers.batch_normalization(h_conv2_2)

    W_conv2_3 = tf.get_variable("W_conv2_3", shape=[3, 3, 64, 64],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv2_3 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2_3 = tf.nn.elu(tf.nn.conv2d(h_bn2_2, W_conv2_3, strides=[1, 1, 1, 1], padding='SAME') + b_conv2_3)
    h_bn2_3 = tf.layers.batch_normalization(h_conv2_3)

    # 두번째 pooling layer.
    h_pool2 = tf.nn.max_pool(h_bn2_3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    h_pool2_drop = tf.nn.dropout(h_pool2, keep_prob) 


    # 세번째 convolutional layer
    #W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
    W_conv3 = tf.get_variable("W_conv3", shape=[3, 3, 64, 128],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3 = tf.nn.elu(tf.nn.conv2d(h_pool2_drop, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)
    h_bn3 = tf.layers.batch_normalization(h_conv3)

    #W_conv3_2 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
    W_conv3_2 = tf.get_variable("W_conv3_2", shape=[3, 3, 128, 128],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv3_2 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3_2 = tf.nn.elu(tf.nn.conv2d(h_bn3, W_conv3_2, strides=[1, 1, 1, 1], padding='SAME') + b_conv3_2)
    h_bn3_2 = tf.layers.batch_normalization(h_conv3_2)
    
    W_conv3_3 = tf.get_variable("W_conv3_3", shape=[3, 3, 128, 128],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv3_3 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3_3 = tf.nn.elu(tf.nn.conv2d(h_bn3_2, W_conv3_3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3_3)
    h_bn3_3 = tf.layers.batch_normalization(h_conv3_3)

    #h_pool3 = tf.nn.max_pool(h_bn3_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    h_pool3_drop = tf.nn.dropout(h_bn3_3, keep_prob) 
    

    # 네번째 convolutional layer
    #W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 256], stddev=5e-2))
    W_conv4 = tf.get_variable("W_conv4", shape=[3, 3, 128, 256],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv4 = tf.Variable(tf.constant(0.1, shape=[256])) 
    h_conv4 = tf.nn.elu(tf.nn.conv2d(h_pool3_drop, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)
    h_bn4 = tf.layers.batch_normalization(h_conv4)

    # 다섯번째 convolutional layer
    #W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 256, 256], stddev=5e-2))
    W_conv5 = tf.get_variable("W_conv5", shape=[1, 1, 256, 256],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv5 = tf.Variable(tf.constant(0.1, shape=[256]))
    h_conv5 = tf.nn.elu(tf.nn.conv2d(h_bn4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)
    h_bn5 = tf.layers.batch_normalization(h_conv5)
    
    W_conv5_2 = tf.get_variable("W_conv5_2", shape=[1, 1, 256, 256],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_conv5_2 = tf.Variable(tf.constant(0.1, shape=[256]))
    h_conv5_2 = tf.nn.elu(tf.nn.conv2d(h_bn5, W_conv5_2, strides=[1, 1, 1, 1], padding='SAME') + b_conv5_2)
    h_bn5_2 = tf.layers.batch_normalization(h_conv5_2)

    h_bn5_drop = tf.nn.dropout(h_bn5_2, keep_prob) 

    # Fully Connected Layer 1 - 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 됩니다.
    # 이를 384개의 특징들로 맵핑(maping)합니다.
    #W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 256, 1024], stddev=5e-2))
    W_fc1 = tf.get_variable("W_fc1", shape=[8 * 8 * 256, 1024],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))

    h_conv5_flat = tf.reshape(h_bn5_drop, [-1, 8*8*256])
    h_fc1 = tf.nn.elu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

    # Dropout - 모델의 복잡도를 컨트롤합니다. 특징들의 co-adaptation을 방지합니다.
    h_fc1_bn = tf.layers.batch_normalization(h_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1_bn, keep_prob) 

    # Fully Connected Layer 2 - 384개의 특징들(feature)을 10개의 클래스-airplane, automobile, bird...-로 맵핑(maping)합니다.
    #W_fc2 = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=5e-2))
    W_fc2 = tf.get_variable("W_fc2", shape=[1024, 10],  initializer=tf.contrib.layers.xavier_initializer(), regularizer=tf.contrib.layers.l2_regularizer(weight_decay))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

In [ ]:
# 인풋 아웃풋 데이터, 드롭아웃 확률을 입력받기 위한 플레이스홀더를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

# CIFAR-10 데이터를 다운로드하고 데이터를 불러옵니다.
(x_train, y_train), (x_test, y_test) = load_data()
# scalar 형태의 레이블(0~9)을 One-hot Encoding 형태로 변환합니다.
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

# Convolutional Neural Networks(CNN) 그래프를 생성합니다.
y_pred, logits = build_CNN_classifier(x)

# Cross Entropy를 비용함수(loss function)으로 정의하고, RMSPropOptimizer를 이용해서 비용 함수를 최소화합니다.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

# 정확도를 계산하는 연산을 추가합니다.
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call in

In [ ]:
#모델 저장 위치
save_file = './model'

saver = tf.train.Saver()

In [ ]:
# 세션을 열어 실제 학습을 진행합니다.
with tf.Session() as sess:
    # 모든 변수들을 초기화한다. 
    sess.run(tf.global_variables_initializer())
    
    # 10000 Step만큼 최적화를 수행합니다.
    for i in range(10000):
        batch = next_batch(128, x_train, y_train_one_hot.eval())

        # 100 Step마다 training 데이터셋에 대한 정확도와 loss를 출력합니다.
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
            loss_print = loss.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})

            print("Epoch: %d, Training Accuracy: %f, Loss: %f" % (i, train_accuracy, loss_print))
        # 20% 확률의 Dropout을 이용해서 학습을 진행합니다.
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1], keep_prob: 0.7})

    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

    # 학습이 끝나면 테스트 데이터(10000개)에 대한 정확도를 출력합니다.    
    test_accuracy = 0.0    
    for i in range(10):
        test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(feed_dict={x: test_batch[0], y: test_batch[1], keep_prob: 1.0})
    test_accuracy = test_accuracy / 10;
    print("Test data Accuracy: %f" % test_accuracy)

Epoch: 0, Training Accuracy: 0.062500, Loss: 2.668514
Epoch: 100, Training Accuracy: 0.062500, Loss: 2.393440
Epoch: 200, Training Accuracy: 0.101562, Loss: 2.374321
Epoch: 300, Training Accuracy: 0.109375, Loss: 6.165642
Epoch: 400, Training Accuracy: 0.132812, Loss: 2.338518
Epoch: 500, Training Accuracy: 0.070312, Loss: 2.376593
Epoch: 600, Training Accuracy: 0.070312, Loss: 2.444928
Epoch: 700, Training Accuracy: 0.101562, Loss: 2.459088
Epoch: 800, Training Accuracy: 0.054688, Loss: 2.617356
Epoch: 900, Training Accuracy: 0.062500, Loss: 2.477672
Epoch: 1000, Training Accuracy: 0.062500, Loss: 2.426536
Epoch: 1100, Training Accuracy: 0.046875, Loss: 2.486279
Epoch: 1200, Training Accuracy: 0.054688, Loss: 2.379369


KeyboardInterrupt: ignored

In [ ]:
def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
label_names = load_label_names()
with tf.Session() as sess:
    saver.restore(sess, save_file)    
    test_batch = next_batch(1, x_test, y_test_one_hot.eval())

    print("Prediction:", label_names[sess.run(tf.argmax(y_pred, 1), {x: test_batch[0], y: test_batch[1], keep_prob: 1.0})[0]])

    # Selected sample showing
    plt.imshow(test_batch[0].reshape(32,32,3), interpolation='nearest')
    plt.show()